In [ ]:
import pandas as pd
import requests
import json
import os
from datetime import datetime, timedelta
from tqdm import tqdm

import FinanceDataReader as fdr

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ────────────────────────────────────────
# 설정
START_DATE = '2025-05-25'
END_DATE = '2025-06-27'
SYMBOL = 'SPY'
OUTPUT_DIR_OPT = './SPY_option_data'
OUTPUT_DIR_VOL = './SPY_option_vol_data'
OUTPUT_DIR_SPOT = './SPY_SPOT_df.csv'
OUTPUT_DIR_GEX = './SPY_GEX_df.csv'

# ────────────────────────────────────────
# 디렉토리 준비
os.makedirs(OUTPUT_DIR_OPT, exist_ok=True)
os.makedirs(OUTPUT_DIR_VOL, exist_ok=True)

# ────────────────────────────────────────
# 날짜 리스트 생성
def generate_date_list(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    date_list = []
    while start <= end:
        date_list.append(start.strftime('%Y-%m-%d'))
        start += timedelta(days=1)
    return date_list

# ────────────────────────────────────────
# API 요청 + 저장
def fetch_and_save(OUTPUT_DIR, date_str, type_="option_chain"):
    # 저장할 파일 경로
    file_path = os.path.join(OUTPUT_DIR, f"{date_str}.json")
    # 이미 있으면 건너뛰기
    if os.path.exists(file_path):
        print(f"[{date_str}] 정보 이미 있음! 건너뜁니당")
        return False

    base_url = 'https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master'
    sql_query = f"select * from {type_} where act_symbol = '{SYMBOL}' and date = '{date_str}'"
    params = {'q': sql_query}

    try:
        resp = requests.get(base_url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        
        # 성공한 경우만 rows가 있으면 저장
        if 'rows' in data and data['rows']:
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            with open(file_path, 'w') as f:
                json.dump(data['rows'], f, indent=2)
            print(f"[{date_str}] saved ({len(data['rows'])} rows).")
            return True
        else:
            print(f"[{date_str}] no data.")
            return False

    except Exception as e:
        print(f"[{date_str}] Error: {e}")
        return False


In [ ]:
# ────────────────────────────────────────
# 메인 루프 (옵션/변동성 데이터 수집)
if __name__ == "__main__":
    date_list = generate_date_list(START_DATE, END_DATE)
    print(f"총 {len(date_list)}일 데이터 요청 시작...")

    for date in tqdm(date_list):
        success_OPT = fetch_and_save(OUTPUT_DIR_OPT, date)
        success_VOL = fetch_and_save(OUTPUT_DIR_VOL, date, 'volatility_history')
        if not success_OPT or not success_VOL:
            # 데이터 없거나 에러인 경우 pass
            continue

    print(f"✅ 데이터 수집 완료!")

In [ ]:
# ────────────────────────────────────────
# 현물 데이터 수집
price_df = fdr.DataReader('YAHOO:SPY')
price_df.to_csv(OUTPUT_DIR_SPOT)

In [ ]:
# ────────────────────────────────────────
# GEX 데이터 수집
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def fetch_gamma_exposure():
    url = "https://unusualwhales.com/stock/SPY/greek-exposure?tab=Gamma"

    # 크롬 드라이버 옵션
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # 헤드리스 실행 시 주석 해제
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 15)

    # 페이지 접속 및 로드 대기
    driver.get(url)
    time.sleep(10)

    # Daily Gamma Exposure Table 그리드 로드
    grid_xpath = (
        "//div[contains(@class,'card_cardTitleText')"
        " and normalize-space(text())='Daily Gamma Exposure Table - SPY']"
        "/ancestor::section//div[@role='grid']"
    )
    grid = wait.until(EC.presence_of_element_located((By.XPATH, grid_xpath)))

    # 스크롤 및 파싱 설정
    seen_dates = set()
    records = []
    scroll_increment = 300  # 픽셀 단위 스크롤 양

    # 전체 스크롤 영역 크기
    scroll_height = driver.execute_script("return arguments[0].scrollHeight", grid)
    scroll_position = 0

    # 반복해서 스크롤하며 데이터 파싱
    while scroll_position <= scroll_height:
        rows = grid.find_elements(By.XPATH, ".//div[@role='row' and not(contains(@class,'rdg-header-row'))]")
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//div[@role='gridcell']")
            if len(cells) < 5:
                continue
            date = cells[0].text.strip()
            if date not in seen_dates:
                seen_dates.add(date)
                records.append({
                    "Date": date,
                    "Call GEX": cells[1].text.strip(),
                    "Put GEX": cells[2].text.strip(),
                    "Net GEX": cells[3].text.strip(),
                    "P/C GEX": cells[4].text.strip()
                })
        # 스크롤 다운
        scroll_position += scroll_increment
        driver.execute_script("arguments[0].scrollTop = arguments[1]", grid, scroll_position)
        time.sleep(1)
        scroll_height = driver.execute_script("return arguments[0].scrollHeight", grid)

    # DataFrame 생성 및 인덱스 설정
    df = pd.DataFrame(records)
    df = df.drop_duplicates(subset=["Date"])  # 중복 제거
    df = df.sort_values(by="Date", ascending=False)
    df.set_index("Date", inplace=True)

    driver.quit()
    return df

if __name__ == "__main__":
    df = fetch_gamma_exposure()
    df.to_csv(OUTPUT_DIR_GEX)